In [12]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
!pip install evaluate --upgrade
!pip install scikit-learn
import evaluate
from peft import get_peft_model, LoraConfig, TaskType

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 5.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:00:01


In [26]:
# Load the emotion dataset
dataset = load_dataset("emotion")

# Load the tokenizer for the model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply the tokenizer to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


/Users/nidhipatel/Documents/DentaFlex/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [27]:
# Load the base model for sequence classification
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Define accuracy metric
accuracy_metric = evaluate.load("accuracy")

# Prepare the dataset for Trainer
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
# Training arguments for evaluation
training_args = TrainingArguments(
    output_dir="./results_base_model",
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    do_train=False,
    do_eval=True
)

# Trainer for the base model
trainer = Trainer(
    model=base_model,
    args=training_args,
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=lambda p: accuracy_metric.compute(predictions=torch.argmax(torch.tensor(p.predictions), axis=-1), references=p.label_ids)
)

# Evaluate the base model
print("Evaluating the base model...")
base_results = trainer.evaluate()
print(f"Base Model Accuracy: {base_results['eval_accuracy']}")



Evaluating the base model...


Base Model Accuracy: 0.0645


In [35]:
# Select a smaller subset for quick fine-tuning
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))

# Define LoRA configuration for PEFT
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
     target_modules=["q_lin", "v_lin"]
)

# Apply PEFT (LoRA) to the base model
peft_model = get_peft_model(base_model, peft_config)

# Training arguments for fine-tuning with PEFT
fine_tune_args = TrainingArguments(
    output_dir="./results_peft_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    logging_dir="./logs_peft"
)

# Trainer for the fine-tuned PEFT model
peft_trainer = Trainer(
    model=peft_model,
    args=fine_tune_args,
    train_dataset=small_train_dataset,
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=lambda p: accuracy_metric.compute(predictions=torch.argmax(torch.tensor(p.predictions), axis=-1), references=p.label_ids)
)

# Fine-tune the PEFT model
print("Fine-tuning the PEFT model...")
peft_trainer.train()


Fine-tuning the PEFT model...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.588295,0.348000
2,1.659300,1.542570,0.374500
3,1.659300,1.531002,0.407000


TrainOutput(global_step=750, training_loss=1.6326583658854166, metrics={'train_runtime': 470.9005, 'train_samples_per_second': 3.185, 'train_steps_per_second': 1.593, 'total_flos': 202137458688000.0, 'train_loss': 1.6326583658854166, 'epoch': 3.0})

In [36]:
# Evaluate the fine-tuned PEFT model
print("Evaluating the fine-tuned PEFT model...")
peft_results = peft_trainer.evaluate()
print(f"PEFT Model Accuracy: {peft_results['eval_accuracy']}")


Evaluating the fine-tuned PEFT model...


PEFT Model Accuracy: 0.407


In [37]:
# Compare the base model and fine-tuned PEFT model results
print("\nComparison of Base and Fine-tuned Model:")
print(f"Base Model Accuracy: {base_results['eval_accuracy']}")
print(f"PEFT Model Accuracy: {peft_results['eval_accuracy']}")



Comparison of Base and Fine-tuned Model:
Base Model Accuracy: 0.0645
PEFT Model Accuracy: 0.407
